In [1]:
import requests
from bs4 import BeautifulSoup
import pandas
import time
import json
import pandas as pd
import re

def change_time(item):
    t=int(item)
    t=time.localtime(t)
    return time.strftime("%Y-%m-%d",t)

In [ ]:
#註記:聽說每個ip爬超過1000則文章就會被中斷

In [ ]:
# articles        | comments        |replies
# -------------------------------------------------------               
# title        N  | id           W  |
# author_name  Y  | comment_id   Y  |
# author_id    Y  | author_name  Y  |
# context      Y  | context      Y  |
# platform_id  N  | post_id      W  |
# post_id      Y  | post_time    Y  |
# post_time    Y  | platfrom_id  N  |
# like_num     Y  | like_num     Y  |
    #haha      Y  |    haha      Y  |
    #anger     Y  |    anger     Y  |
    #love      Y  |    love      Y  |
    #sorry     Y  |    sorry     Y  |
    #wow       Y  |    wow       Y  |
# share_num    Y  |

# ------------------------------------------------------- 
 #comment_count(留言+回覆)
 #display_commnets(留言)
 #如果硬要的話其實也可以將p標籤第一句作為標題
    
#初始url='https://www.facebook.com/pages_reaction_units/more/?page_id=433004610088636&cursor={"card_id":"page_posts_divider","has_next_page":true}&surface=www_pages_home&unit_count=8&fb_dtsg_ag&__user=0&__a=1'
#透過正則表達式抓取網址內容中的cursor，並只擷取有關timeline_cursor的部分
#域名+page_id+timeline_cursor+&surface=www_pages_home&unit_count=8&fb_dtsg_ag&__user=0&__a=1
#unit_count 可以替換

#註記:好像會少最新兩則

In [116]:
headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'}
#url='https://www.facebook.com/weather.taiwan/?hc_ref=ARQCAALTgri66IJDHd1vIrLquvP0gMsmfAwC6t9g1fhyVI3yaz1XEiduOIufIIZfBDk&fref=nf&__tn__=kC-R'
url='https://www.facebook.com/CWB.TW/'
resp=requests.get(url,headers=headers)
pattern=r'"entity_id":"\d+"'
page_id=re.search(pattern,resp.text).group()
pattern=r'\d+'
page_id=re.search(pattern,page_id).group()
page_id

'129328600420380'

In [79]:
data_text=[]
data_time=[]
err_data=[]
data_feedback=[]

In [ ]:
#文章
#url='https://www.facebook.com/pages_reaction_units/more/?page_id=433004610088636&cursor={%22card_id%22:%22page_posts_divider%22,%22has_next_page%22:true}&surface=www_pages_home&unit_count=10&fb_dtsg_ag&__user=0&__a=1'
#url='https://www.facebook.com/pages_reaction_units/more/?page_id=433004610088636&cursor={%22timeline_cursor%22:%22AQHRxd9iiqy86qLrSw_yb_moxoNysqr-e2zPn91UGJv2Be6bTAIzKoDQI5nY49eMcRKk-yl7fjStyO7pH1kPl-Em1ygmVyKQI3r-JmAeItN2VHURii3-m_qJyVgr-QX_9pbZ%22,%22timeline_section_cursor%22:{},%22has_next_page%22:true}&surface=www_pages_home&unit_count=8&fb_dtsg_ag&__user=0&__a=1'
url='https://www.facebook.com/pages_reaction_units/more/?page_id=433004610088636&cursor={%22timeline_cursor%22:"",%22timeline_section_cursor%22:{},%22has_next_page%22:true}&surface=www_pages_home&unit_count=8&fb_dtsg_ag&__user=0&__a=1'


for i in range(1):
    #try:
    session=requests.session()
    resp=session.get(url)
    #----------likenum and share_num-------
    data=json.loads(resp.text.split(';',3)[3])['jsmods']['pre_display_requires']
    dict_for_temp={}#暫時儲存特定post_id的likenum and share_num

    for i in range(0,len(data)):
        try:
            post_id=data[i][3][1]['__bbox']['result']['data']['feedback']['subscription_target_id']
            feedback_id=data[i][3][1]['__bbox']['result']['data']['feedback']['id']
            reaction_dict={}
            reaction_data=data[i][3][1]['__bbox']['result']['data']['feedback']['top_reactions']['edges']
            for j in range(0,len(reaction_data)):
                reaction_type=reaction_data[j]['node']['reaction_type']
                reaction_count=reaction_data[j]['reaction_count']
                reaction_dict[reaction_type]=int(reaction_count)
                #總共六種心情
                share_num=data[i][3][1]['__bbox']['result']['data']['feedback']['share_count']['count']
                reaction_dict['share_num']=share_num
                reaction_dict['feedback_id']=feedback_id
                data_feedback.append(feedback_id)#delete
            dict_for_temp[post_id]=reaction_dict
        except:
            pass
    #-----------post_time,context and post_id-------------
    texts=json.loads(resp.text.split(';',3)[3])['domops'][0][3]['__html']
    soup=BeautifulSoup(texts,'lxml')
    items=soup.find_all('div',class_="_4-u2 _4-u8")
    url_section=soup.find('div',class_="clearfix uiMorePager stat_elem _52jv").a['ajaxify']

    for item in items:

        context=item.find('p').text
        post_time=change_time(item.find('abbr',class_="_5ptz")['data-utime'])
        temp=str(item.find('a',class_="see_more_link"))
        pattern=r'posts/\d+"'
        temp=re.search(pattern,temp).group()
        pattern='\d+'
        post_id=re.search(pattern,temp).group()
        print('post_id:'+post_id)
        print(post_time)
        print(context)
        print(dict_for_temp[post_id])
        data_text.append(context)
        data_time.append(post_time)
        print('--------------------')
    time.sleep(5)
    url='https://www.facebook.com'+url_section+'&fb_dtsg_ag&__user=0&__a=1'
    #except:
    #    err_data.append(item)
    #    print('error')



In [11]:
article_data=pd.DataFrame()
article_data['context']=data_text
article_data['post_time']=data_time
article_data['author_name']='天氣即時預報'

In [84]:
data_feedback=list(set(data_feedback))

In [ ]:
#找取其他相關資料，如:feedtargetid
json.loads(resp.text.split(';',3)[3])['jsmods']

In [176]:
data_con=[]
data_texts=[]
params_0={'__user': '0',
          '__a': '1',
          "doc_id": "2808628465838895",
          'variables':'''{
          "first":50,
          "feedbackID":"",
          "viewOption":"RANKED_UNFILTERED",
          "after":null
         }'''
         }
if 'null' in params_0['variables']:
    params_0['variables']=params_0['variables'].replace('null','None')
test=eval(params_0['variables'])
test['feedbackID']="ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMQ=="
params_0['variables']=str(test)
params_0['variables']=params_0['variables'].replace('None','null')

#while True:
for i in range(2):
    url='https://www.facebook.com/api/graphql/'
    session=requests.session()
    session.headers['User-Agent']='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
    resp=session.post(url,data=params_0)

    data=json.loads(resp.text)
    time.sleep(5)
    post_id=data['data']['feedback']['id']#post_id
    end_cursor=data['data']['feedback']['display_comments']['page_info']['end_cursor']
    for j in range(0,len(data['data']['feedback']['display_comments']['edges'])):
        try:
            post_time=change_time(data['data']['feedback']['display_comments']['edges'][j]['node']['created_time'])#post_time
            #author_id=data['data']['feedback']['display_comments']['edges'][j]['node']['author']['id']#author_id
            author_name=data['data']['feedback']['display_comments']['edges'][j]['node']['author']['name']#author_name    
            feedback_id=data['data']['feedback']['display_comments']['edges'][j]['node']['feedback']['id']#找取回覆的查詢參數
            comment_id=data['data']['feedback']['display_comments']['edges'][0]['node']['legacy_fbid']
            reaction_dict={}
            reaction_data=data['data']['feedback']['display_comments']['edges'][j]['node']['feedback']['top_reactions']['edges']#心情
            for i in range(0,len(reaction_data)):
                reaction_type=reaction_data[i]['node']['reaction_type']
                reaction_count=reaction_data[i]['reaction_count']
                reaction_dict[reaction_type]=reaction_count
            #共六種心情

            reply_count=data['data']['feedback']['display_comments']['edges'][j]['node']['feedback']['comment_count']['total_count']#回覆數
            context=data['data']['feedback']['display_comments']['edges'][j]['node']['preferred_body']['text']#context
            if reply_count>=1:
                data_con.append(feedback_id)
            data_texts.append(context)
            #print('flatform_id:'+self.name)
            print('post_id:'+post_id)
            print('comment_id:'+comment_id)
            print('post_time:'+post_time)
            #print('author_id:'+author_id)
            print('author_name:'+author_name)
            print('context:'+context)
            print('reply_count:'+str(reply_count))#if reply_count>0:run reply_parse
            print(reaction_dict)
            print('-------------------------------------------------')
        except:
            pass

    if end_cursor ==None:
        break
    else:
        if 'null' in params_0['variables']:
            params_0['variables']=params_0['variables'].replace('null','None')
        test=eval(params_0['variables'])
        test['after']=end_cursor
        params_0['variables']=str(test)
        params_0['variables']=params_0['variables'].replace('None','null')

post_id:ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMQ==
comment_id:2908475872541485
post_time:2020-04-09
author_name:范安室
context:請問冷氣團是全台都冷嗎？周一要前往台中
reply_count:8
{}
-------------------------------------------------
post_id:ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMQ==
comment_id:2908475872541485
post_time:2020-04-09
author_name:Chen Amy
context:小編之前說冬衣可以收一半起來了
我買好整理箱要收的說 🤣🤣🤣
reply_count:8
{'LIKE': 1}
-------------------------------------------------
post_id:ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMQ==
comment_id:2908475872541485
post_time:2020-04-09
author_name:張玉玲
context:拜託千萬別太冷啊！更不要下雨喔！下星期一.星期二要回.最愛台中放輕鬆.離開恐怖冷漠北部人啊！
reply_count:4
{'LIKE': 3, 'HAHA': 3}
-------------------------------------------------
post_id:ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMQ==
comment_id:2908475872541485
post_time:2020-04-09
author_name:Anna Shao
context:四月居然有冷氣團，印象中沒遇過
reply_count:18
{'LIKE': 33, 'ANGER': 1}
-------------------------------------------------
post_id:ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMQ==
comment_id:2908475872541485
post_time:

post_id:ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMQ==
comment_id:2908493435873062
post_time:2020-04-09
author_name:Jessie Tsai
context:翁小慧媽呀，星期日會冷又下雨耶
reply_count:1
{'LIKE': 1}
-------------------------------------------------
post_id:ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMQ==
comment_id:2908493435873062
post_time:2020-04-09
author_name:水晨凱
context:看來還是再入一桶煤油好了
reply_count:0
{}
-------------------------------------------------
post_id:ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMQ==
comment_id:2908493435873062
post_time:2020-04-09
author_name:Grace Huang
context:嫈琦竟然有強烈冷氣團!
reply_count:1
{}
-------------------------------------------------
post_id:ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMQ==
comment_id:2908493435873062
post_time:2020-04-09
author_name:黃雯倩
context:還好厚被沒收
reply_count:0
{}
-------------------------------------------------
post_id:ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMQ==
comment_id:2908493435873062
post_time:2020-04-09
author_name:郭芳如
context:請問圖片是最低溫嗎？
屆時高雄的最高溫大約落在幾度呢？
reply_count:0
{}
------------------------

In [220]:
data_reply=[]
for t in ['ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMV8yOTA5MDQ4ODI5MTUwODU2']:
    params_0={'__user': '0',
                  '__a': '1',
                  "doc_id": "2808628465838895",
                  'variables':'''{
                  "first":50,
                  "feedbackID":"",
                  "viewOption":"RANKED_UNFILTERED",
                  "after":null,
                  "UFI2CommentsProvider_commentsKey":null,
                 }'''
                 }
    if 'null' in params_0['variables']:
        params_0['variables']=params_0['variables'].replace('null','None')
    if 'false' in params_0['variables']:
        params_0['variables']=params_0['variables'].replace('false','False')
    test=eval(params_0['variables'])
    test['feedbackID']=t
    params_0['variables']=str(test)
    params_0['variables']=params_0['variables'].replace('None','null')

    #while True:
    for w in range(1):
        url='https://www.facebook.com/api/graphql/'
        session=requests.session()
        session.headers['User-Agent']='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
        resp=session.post(url,data=params_0)

        data=json.loads(resp.text)
        time.sleep(2)
        post_id=data['data']['feedback']['id']#post_id
        end_cursor=data['data']['feedback']['display_comments']['page_info']['end_cursor']
        for j in range(0,len(data['data']['feedback']['display_comments']['edges'])):
            try:
                post_time=change_time(data['data']['feedback']['display_comments']['edges'][j]['node']['created_time'])#post_time
                #author_id=data['data']['feedback']['display_comments']['edges'][j]['node']['author']['id']#author_id
                author_name=data['data']['feedback']['display_comments']['edges'][j]['node']['author']['name']#author_name    
                #data['data']['feedback']['display_comments']['edges'][j]['node']['feedback']['id']#找取回覆的查詢參數
                comment_id=data['data']['feedback']['display_comments']['edges'][0]['node']['legacy_fbid']
                reaction_dict={}
                reaction_data=data['data']['feedback']['display_comments']['edges'][j]['node']['feedback']['top_reactions']['edges']#心情
                for i in range(0,len(reaction_data)):
                    reaction_type=reaction_data[i]['node']['reaction_type']
                    reaction_count=reaction_data[i]['reaction_count']
                    reaction_dict[reaction_type]=reaction_count
                #共六種心情

                reply_count=data['data']['feedback']['display_comments']['edges'][j]['node']['feedback']['comment_count']['total_count']#回覆數
                context=data['data']['feedback']['display_comments']['edges'][j]['node']['preferred_body']['text']#context
                data_reply.append(context)
                #print('flatform_id:'+self.name)
                print('post_id:'+post_id)
                print('comment_id:'+comment_id)
                print('post_time:'+post_time)
                #print('author_id:'+author_id)
                print('author_name:'+author_name)
                print('context:'+context)
                print('reply_count:'+str(reply_count))#if reply_count>0:run reply_parse
                print(reaction_dict)
                print('-------------------------------------------------')
            except:
                pass

        if end_cursor ==None:
            break
        else:
            if 'null' in params_0['variables']:
                params_0['variables']=params_0['variables'].replace('null','None')
            if 'false' in params_0['variables']:
                params_0['variables']=params_0['variables'].replace('false','False')
            test=eval(params_0['variables'])
            test['after']=end_cursor
            test["UFI2CommentsProvider_commentsKey"]=test['feedbackID']
            params_0['variables']=str(test)
            params_0['variables']=params_0['variables'].replace('None','null')

post_id:ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMV8yOTA5MDQ4ODI5MTUwODU2
comment_id:2913433818712357
post_time:2020-04-11
author_name:Eason Shen
context:周鈺婷 走啊 去哪
reply_count:0
{}
-------------------------------------------------
post_id:ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMV8yOTA5MDQ4ODI5MTUwODU2
comment_id:2913433818712357
post_time:2020-04-11
author_name:周鈺婷
context:歐鈺穗 出去玩ㄌ
reply_count:0
{}
-------------------------------------------------
post_id:ZmVlZGJhY2s6MjkwODQ2ODE3OTIwODkyMV8yOTA5MDQ4ODI5MTUwODU2
comment_id:2913433818712357
post_time:2020-04-09
author_name:歐鈺穗
context:Eason Shen 
reply_count:2
{}
-------------------------------------------------


In [207]:
post_id=data['data']['feedback']['display_comments']['edges'][0]['node']['legacy_token']#.split('_')[0]
post_id

'2908468179208921_2908829492506123'

In [ ]:
#流程:
#1.從首頁網址獲取page_id作為查詢文章的網址中的參數之一，並利用該網址獲取ㄧjson檔，裡面有一部分為script標籤中的內容(即為文章內容，如:post_time,author_id,author_name,context)
#2.除了利用上述網址獲取文章內容，也可以獲取like_num,share_num,feeback_id(作為獲取某文章下的留言的查詢參數) and post_id
#2.1 #註記:like_num等資料是需要以post_id作為key來獲取的
#----------------以上為文章部分---------------
#3.利用得到的feedback_id以及一些參數作為post給留言查詢網址的查詢參數(重要參數如_user,_a,feedback_id,view_option(排序方式),after(指向下一篇文章的參數))
#4.利用上述網址或的的json檔來解析留言資料表所需的內容，基本上與文章的解析差不多
#----------------以上為留言部分----------------
#5. #註記:回覆部分跟留言部分一模一樣只是改掉了feedback_id而已
#----------------以上為回覆部分----------------
#因為我們需要替換feedback_id與after，所以在爬取的參數variables進行替換但是該參數是以字串作為傳遞的型態，單純使用正規表達式太麻煩，所以我改成替換時需要暫時轉成字典型態(eval())，
#但是裡面會有null的出現，需要先將其替換成None，替換完feedback_id或after時，需要再將variables的型態轉為字串並且以None替換回去null


In [290]:
import pandas as pd
data=pd.read_csv('tutorial/article_ettoday.csv')

In [70]:
data

,anger_num,author_id,author_name,comment_id,context,haha_num,like_num,love_num,platform_id,post_id,post_time,sorry_num,wow_num
0,NaN,100000667791375,范安室,2908475872541485,請問冷氣團是全台都冷嗎？周一要前往台中,NaN,NaN,NaN,facebook,2908468179208921,2020-04-09,NaN,NaN
1,NaN,100000138110237,Chen Amy,2908539849201754,小編之前說冬衣可以收一半起來了\n我買好整理箱要收的說 🤣🤣🤣,NaN,1.0,NaN,facebook,2908468179208921,2020-04-09,NaN,NaN
2,NaN,100023559169189,張玉玲,2908471202541952,拜託千萬別太冷啊！更不要下雨喔！下星期一.星期二要回.最愛台中放輕鬆.離開恐怖冷漠北部人啊！,3.0,3.0,NaN,facebook,2908468179208921,2020-04-09,NaN,NaN
3,1.0,100011199960983,Anna Shao,2908512175871188,四月居然有冷氣團，印象中沒遇過,3.0,33.0,NaN,facebook,2908468179208921,2020-04-09,NaN,NaN
4,1.0,100000116677212,陳潔慧,2909840275738378,因北方仍有大陸冷高壓帶下東北風之故，今天(9日)台中有稍涼的北風，且雲少天晴中午有27~28...,3.0,2.0,1.0,facebook,2908468179208921,2020-04-10,NaN,NaN
5,1.0,100002311894503,Peggy Lee,2908470329208706,請問台南星期日會下很大嗎？要出去 🤧,3.0,2.0,1.0,facebook,2908468179208921,2020-04-09,NaN,NaN
6,1.0,100012557554243,郭芳菻,2908483642540708,請問禮拜六大約晚上下雨還是中午過後就開始下大雨了呢,3.0,2.0,1.0,facebook,2908468179208921,2020-04-09,NaN,NaN
7,1.0,100013576564225,劉俊賢,2908471349208604,還有冷氣團 會很冷嗎,3.0,5.0,1.0,facebook,2908468179208921,2020-04-09,NaN,NaN
8,1.0,100007819485237,廖婉鈴,2908477392541333,臺中是早晚冷嗎,3.0,1.0,1.0,facebook,2908468179208921,2020-04-09,NaN,NaN
9,1.0,100001254616304,Liu Juke,2908589672530105,今天台南天氣溫暖涼爽還算舒適，謝謝小編早安😀,3.0,1.0,1.0,facebook,2908468179208921,2020-04-09,NaN,NaN
